In [1]:
import matplotlib.pyplot as plt
import scipy.integrate
import torch
import sys  
sys.path.insert(0, '..')

from hnn.simulation import get_dataset, get_field, get_trajectory, get_vector_field, integrate_model
from hnn.train import train

In [2]:
import argparse

def get_args():
    parser = argparse.ArgumentParser(allow_abbrev=False)
    parser.add_argument('--input_dim', default=2, type=int, help='dimensionality of input tensor')
    parser.add_argument('--hidden_dim', default=200, type=int, help='hidden dimension of mlp')
    parser.add_argument('--learn_rate', default=1e-3, type=float, help='learning rate')
    parser.add_argument('--total_steps', default=2000, type=int, help='number of gradient steps')
    parser.add_argument('--name', default='pend', type=str, help='only one option right now')
    parser.add_argument('--field_type', default='solenoidal', type=str, help='type of vector field to learn') # solenoidal, conservative
    parser.set_defaults(feature=True)
    return parser.parse_known_args()[0]

In [3]:
args = get_args()
model, stats = train(args)

OK FFJJFJF conservative
X shape torch.Size([50, 90, 2]) DX shape torch.Size([50, 90, 2])
FIELD TYPE conservative
FIELD TYPE conservative
step 0, train_loss 6.1895e+00, test_loss 5.8859e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 1, train_loss 6.1687e+00, test_loss 5.8982e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 2, train_loss 6.1794e+00, test_loss 5.8884e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 3, train_loss 6.1704e+00, test_loss 5.8781e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 4, train_loss 6.1608e+00, test_loss 5.8793e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 5, train_loss 6.1611e+00, test_loss 5.8851e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 6, train_loss 6.1654e+00, test_loss 5.8855e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 7, train_loss 6.1648e+00, test_loss 5.8817e+00
FIELD TYPE conservative
FIELD TYPE conservative
step 8, train_loss 6.1605e+00, test_loss 5.8795e+00
FIELD TYPE 

In [ ]:
R = 2.5
t_span=[0.0, 28.0]
Y0 = torch.tensor([2.1, 0.0])
LINE_SEGMENTS = 10
ARROW_SCALE = 40
ARROW_WIDTH = 6e-3
LINE_WIDTH = 2

field_args = {
    'xmin': -R,
    'xmax': R,
    'ymin': -R,
    'ymax': R,
    'gridsize': 10,
}

In [ ]:

# integrate along those fields starting from point (1,0)
IVP_TIMESCALE = 100
ivp = integrate_model(
        model,
        t_span,
        y0=Y0,
        timescale=IVP_TIMESCALE,
        rtol=1e-12
)

field = get_field(**field_args)
vector_field = get_vector_field(model, **field_args)


In [ ]:
fig = plt.figure(figsize=(11.3, 3.2), facecolor='white', dpi=300)

# plot data
fig.add_subplot(1, 4, 2, frameon=True)
x, y, dx, dy, t = get_trajectory(t_span=t_span, radius=R, y0=Y0)
N = x.shape[1]
point_colors = [(i/N, 0, 1-i/N) for i in range(N)]
plt.scatter(x, y, s=14, label='data', c=point_colors)
plt.quiver(
        field['x'][:,:,0],
        field['x'][:,:,1],
        field['dx'][:,0],
        field['dx'][:,1],
        cmap='gray_r',
        scale=ARROW_SCALE,
        width=ARROW_WIDTH,
        color=(.2,.2,.2)
)  
plt.xlabel("$q$", fontsize=14)
plt.ylabel("$p$", rotation=0, fontsize=14)
plt.title("Data", pad=10)

# plot HNN
fig.add_subplot(1, 4, 4, frameon=True)
plt.quiver(
        field['x'][:,:,0],
        field['x'][:,:,1],
        vector_field[:,:,0],
        vector_field[:,:,1],
        cmap='gray_r',
        scale=ARROW_SCALE,
        width=ARROW_WIDTH,
        color=(.5,.5,.5)
)


lines = round(ivp.shape[0] / 30)

for i, l in enumerate(torch.tensor_split(ivp, lines)):
        color = (float(i)/lines, 0, 1-float(i)/lines)
        plt.plot(l[:, 0], l[:, 1], color=color, linewidth=LINE_WIDTH)

plt.xlabel("$q$", fontsize=14)
plt.ylabel("$p$", rotation=0, fontsize=14)
plt.title("Hamiltonian NN", pad=10)
plt.tight_layout() ; plt.show()